<a href="https://colab.research.google.com/github/Itsuki-Hamano123/ML_DEMO_UI/blob/master/gradio-app/text_model_GiNZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q transformers gradio

In [2]:
%pip show tensorflow
%pip show transformers

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: h5py, termcolor, protobuf, opt-einsum, tensorboard, wrapt, wheel, grpcio, numpy, six, astunparse, tensorflow-estimator, scipy, gast, keras-preprocessing, google-pasta, absl-py
Required-by: fancyimpute
Name: transformers
Version: 3.3.1
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.6/dist-packages
Requires:

### 形態素解析用GiNZA辞書のインストール

In [3]:
%pip install -q ginza
%pip show ginza

import pkg_resources, imp
imp.reload(pkg_resources)

Name: ginza
Version: 4.0.5
Summary: GiNZA, An Open Source Japanese NLP Library, based on Universal Dependencies
Home-page: https://github.com/megagonlabs/ginza
Author: Megagon Labs, Tokyo.
Author-email: ginza@megagon.ai
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: spacy, ja-ginza, SudachiPy, SudachiDict-core
Required-by: 


<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

In [4]:
# spacyとGiNZAの実行確認
import spacy

JA_DICT = 'ja_ginza' #@param {type:'string'}
parser = spacy.load(JA_DICT)
sample_text = "8月3日に放送された「中居正広の金曜日のスマイルたちへ」(TBS系)で、1日たった5分でぽっこりおなかを解消するというダイエット方法を紹介。キンタロー。のダイエットにも密着。"

res = parser(sample_text)
for sent in res.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.i)
    print('EOS')

0 8 8 NUM 名詞-数詞 nummod 3
1 月 月 NOUN 名詞-普通名詞-助数詞可能 compound 3
2 3 3 NUM 名詞-数詞 nummod 3
3 日 日 NOUN 接尾辞-名詞的-助数詞 obl 5
4 に に ADP 助詞-格助詞 case 3
5 放送 放送 VERB 名詞-普通名詞-サ変可能 acl 16
6 さ する AUX 動詞-非自立可能 advcl 5
7 れ れる AUX 助動詞 aux 5
8 た た AUX 助動詞 aux 5
9 「 「 PUNCT 補助記号-括弧開 punct 11
10 中居 中居 PROPN 名詞-固有名詞-人名-姓 compound 11
11 正広 正広 PROPN 名詞-固有名詞-人名-名 nmod 13
12 の の ADP 助詞-格助詞 case 11
13 金曜日 金曜日 NOUN 名詞-普通名詞-副詞可能 nmod 16
14 の の ADP 助詞-格助詞 case 13
15 スマイル スマイル NOUN 名詞-普通名詞-一般 compound 16
16 たち たち NOUN 接尾辞-名詞的-一般 ROOT 16
17 へ へ ADP 助詞-格助詞 case 16
18 」 」 PUNCT 補助記号-括弧閉 punct 16
EOS
19 ( （ PUNCT 補助記号-括弧開 punct 21
20 TBS TBS PROPN 名詞-固有名詞-一般 compound 21
21 系 系 NOUN 接尾辞-名詞的-一般 nmod 40
22 ) ) PUNCT 補助記号-括弧閉 punct 21
23 で で ADP 助詞-格助詞 cop 21
24 、 、 PUNCT 補助記号-読点 punct 21
25 1 1 NUM 名詞-数詞 nummod 26
26 日 日 NOUN 名詞-普通名詞-助数詞可能 nmod 29
27 たった たった ADV 副詞 advmod 29
28 5 5 NUM 名詞-数詞 nummod 29
29 分 分 NOUN 名詞-普通名詞-助数詞可能 obl 33
30 で で ADP 助詞-格助詞 case 29
31 ぽっこりおなか ぽっこりおなか NOUN 名詞-普通名詞-一般 obj 33
32 を を ADP 助詞-格助詞 case 3

In [5]:
def wakati_doc(parser, doc, join_key=' '):
    '''文書データを形態素解析
    
    Parameters
    -----
    - parser: parser_obj
    - doc : string
        形態素解析対象の文章
    - join_key : string (default:' ')
        解析結果を結合する際の文字

    Returns
    -----
    - result
    '''
    vocabes = []
    result = parser(doc)
    for sent in result.sents:
        for token in sent:
            top_pos = _pop_top_tag(tag_res=token.tag_)
            if _is_allow_pos(token_pos=top_pos):
                vocabes.append(token.lemma_)

    result = join_key.join(vocabes)
    return result


def _pop_top_tag(tag_res):
    '''
    解析結果から先頭の品詞のみを抽出
    '''
    
    def _split_tag(split_key='-'):
        '''「品詞大項目-品詞中項目-品詞小項目」の形式を「-」で分割'''
        return tag_res.split('-')

    tags = []
    tags = _split_tag()
    tpo_tag = tags[0]
    return tpo_tag


def _is_allow_pos(token_pos):
    '''
    ストップワードに該当しない品詞の場合にTrueを返す

    Parameters
    -----
    - token_pos : string
        単語の品詞種別

    Returns
    -----
    - allow_pos_flag : boolean
    '''
    allow_pos_flag = True
    if token_pos in STOP_POS_LIST:
        allow_pos_flag = False
    return allow_pos_flag

## デモ用WebUIに必要なモジュールのインポート

In [6]:
import numpy as np

import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead

## 感情分析モデルのデモ用WebUI作成

モデルの選択などは全てpipelineに任せてみる

In [9]:
%%time
taskname = 'sentiment-analysis' #@param{type:'string'}
nlp = pipeline(taskname)

# 排除したいストップワードの品詞を設定
STOP_POS_LIST = ["助詞", "助動詞", "補助記号"]

# WebUIの入出力定義
input_box = 'textbox'
output_def = gr.outputs.Label(num_top_classes=1)


# 入力を受け取ってから行う処理を定義
def sensity_ana_nlp(input_text):
  print(input_text)
  wakati_text = wakati_doc(parser, input_text)
  print(wakati_text)
  result = nlp(wakati_text)[0]
  return {result['label']: float(result['score'])}


# Webページの起動
gr.Interface(fn=sensity_ana_nlp, inputs=input_box, outputs=output_def).launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://12659.gradio.app
Interface loading below...


CPU times: user 2.28 s, sys: 191 ms, total: 2.47 s
Wall time: 7.52 s


## 翻訳(ja→en)モデルのデモ用WebUI作成
使用するモデル:[https://huggingface.co/Helsinki-NLP/opus-mt-ja-en](https://huggingface.co/Helsinki-NLP/opus-mt-ja-en)

In [8]:
%%time
# translation_ja_to_enが無いのでen_to_roを指定
# モデルとトークナイザがあってれば動く模様
taskname = 'translation_en_to_ro' #@param{type:'string'}
specify_hugging_model = 'Helsinki-NLP/opus-mt-ja-en'#@param{type:'string'}

tokenizer = AutoTokenizer.from_pretrained(specify_hugging_model)
model = AutoModelWithLMHead.from_pretrained(specify_hugging_model)
nlp = pipeline(taskname, model=model, tokenizer=tokenizer)


# WebUIの入出力定義
input_box = 'textbox'
output_box = 'textbox'


# 入力を受け取ってから行う処理を定義
def seq2seq_fn(input_text):
  output_key = 'translation_text'
  result = nlp([input_text])[0]
  return result[output_key]


# Webページの起動
gr.Interface(fn=seq2seq_fn, inputs=input_box, outputs=output_box).launch(debug=False)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:785: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://56710.gradio.app
Interface loading below...


CPU times: user 5.31 s, sys: 791 ms, total: 6.1 s
Wall time: 10.6 s
